In [1]:
#%%
!pip install --upgrade pip
!pip install --upgrade ipython jupyter
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install sklearn
!pip install seaborn



Requirement already up-to-date: pip in /srv/conda/envs/notebook/lib/python3.7/site-packages (19.1.1)
Requirement already up-to-date: ipython in /srv/conda/envs/notebook/lib/python3.7/site-packages (7.5.0)
Requirement already up-to-date: jupyter in /srv/conda/envs/notebook/lib/python3.7/site-packages (1.0.0)


In [2]:
import matplotlib.pyplot as plt
from sklearn.datasets import load_wine
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.neural_network import MLPClassifier

from sklearn.preprocessing import StandardScaler

# import pydotplus
from sklearn.model_selection import train_test_split

In [3]:
def line_to_column_test(all_read_parameters):

    test_data = pd.read_csv("winequality-red.csv", sep=';') 

    #X, y = test_data.iloc[:, :-1].values, test_data.iloc[:, -1].values
    #print (X, y, type(y))
    for name_p in all_read_parameters:
        test_data[name_p] = test_data.loc[:, name_p].values
    
    #df.loc[:, 'quality_cat'] = (df.loc[:, 'quality'] > 5).astype(int)
    test_data['quality_cat'] = (test_data.loc[:, 'quality'] > 5).astype(int) # buy wine or no buy
    #print (test_data['quality_cat'])

    return test_data

def line_to_column_train(all_read_parameters):

    train_data = pd.read_csv("winequality-white.csv", sep=';')

    #X, y = train_data.iloc[:, :-1].values, train_data.iloc[:, -1].values
    #print (X, y, type(y))
    for name_p in all_read_parameters:
        train_data[name_p] = train_data.loc[:, name_p].values

    train_data['quality_cat'] = (train_data.loc[:, 'quality'] > 5).astype(int)

    return train_data

def get_data():
    all_parameters = [  
                'fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 
                'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 
                'density', 'pH', 'sulphates', 'alcohol', 'quality'
                ]
    #train_data = pd.read_csv("winequality-white.csv")
    #test_data = pd.read_csv("winequality-red.csv") 
    test_data = line_to_column_test(all_parameters)
    train_data = line_to_column_train(all_parameters)
    all_data = pd.concat([train_data, test_data])

    #next set - MAIN parameters see lab `1`
    predictors = [
                "alcohol",
                "density",
                "chlorides", 
                "volatile acidity",
                'total sulfur dioxide',
                    # 'fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 
                    # 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 
                    # 'density', 'pH', 'sulphates', 'alcohol'
                ]
    scaler = StandardScaler()
    scaler.fit(all_data[predictors])

    train_data_scaled = scaler.transform(train_data[predictors])
    print (train_data.head())
    #print("===========\train_data_scaled\n========\n", train_data_scaled)  
    test_data_scaled = scaler.transform(test_data[predictors])
    #print("===========\test_data_scaled\n========\n", test_data_scaled)  

    return train_data[predictors], train_data["quality_cat"], test_data[predictors], test_data["quality_cat"]  


def print_score(classifier, x_train, x_test, y_train, y_test):
    """ Simple print score function that prints train and test score of classifier - almost not worth it"""
    print('In Sample Score: ',
          classifier.score(x_train, y_train))
    print('Test Score: ',
          classifier.score(x_test, y_test))

      
def train_wine_neural_net(x_train, y_train, hidden_layers=(32, 32, 32)):
    
    classifier = MLPClassifier(early_stopping=True, alpha=1e-5, solver='adam', learning_rate='adaptive', activation='tanh')

#     parameter_space = {
#         # 'hidden_layer_sizes': [(100, 100, 100), (80, 80, 80), (90, 90, 90)],
#         'hidden_layer_sizes': [(100, 100, 100), (100, 100,), (190, 11, 44), (90, 90, 90), (80, 90, 100), (100, 80, 90), (90, 100, 80) ] ,
#         'activation': ['tanh', 'relu', 'logistic', 'identity'], 
#         'solver': ['sgd', 'adam', 'lbfgs'],
#         'alpha': [0.0001, 0.001, 0.005, 0.05, 0.5],
#         'learning_rate': ['constant', 'adaptive'],
#     }

#     clf = GridSearchCV(classifier, parameter_space, n_jobs=-1, cv=3, verbose=1)
#     clf.fit(x_train, y_train)
#     classifier.fit(x_train, y_train)

#     print('Best parameters found:\n', clf.best_params_)

#     means = clf.cv_results_['mean_test_score']
#     stds = clf.cv_results_['std_test_score']
    # for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    #     print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

    #'activation': 'tanh', 'alpha': 0.005, 'hidden_layer_sizes': (100, 100, 100), 'learning_rate': 'adaptive', 'solver': 'adam' = 0.72
    # 'activation': 'relu', 'alpha': 0.05, 'hidden_layer_sizes': (100, 100, 100), 'learning_rate': 'adaptive', 'solver': 'lbfgs'}
    n = MLPClassifier (early_stopping=True,
                       alpha=1e-5,
                       solver='adam',
                       learning_rate='adaptive',
                       activation='tanh',
                       hidden_layer_sizes=(90, 90, 90),
                       verbose=False)
    n.fit(x_train, y_train)
    return n

In [4]:
get_ipython().run_line_magic('matplotlib', 'inline')
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (30, 30)

## Get Data
x_train, y_train, x_test, y_test = get_data()
print("x_train = \n", x_train.head())
print("y_test = \n", y_test.head())
## try different hidden layer settings, less is probably more
print('\n', 'Neural Net Wine Score:')
mlp = train_wine_neural_net(x_train, y_train)
print_score(mlp, x_train, x_test, y_train, y_test)

   fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0            7.0              0.27         0.36            20.7      0.045   
1            6.3              0.30         0.34             1.6      0.049   
2            8.1              0.28         0.40             6.9      0.050   
3            7.2              0.23         0.32             8.5      0.058   
4            7.2              0.23         0.32             8.5      0.058   

   free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
0                 45.0                 170.0   1.0010  3.00       0.45   
1                 14.0                 132.0   0.9940  3.30       0.49   
2                 30.0                  97.0   0.9951  3.26       0.44   
3                 47.0                 186.0   0.9956  3.19       0.40   
4                 47.0                 186.0   0.9956  3.19       0.40   

   alcohol  quality  quality_cat  
0      8.8        6            1  
1      9.5      